In [1]:
import xml.etree.ElementTree as ET
import glob

In [2]:
import time
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
# we will work with totImages images
totImages = 100

In [5]:
fn = glob.glob("VOC2012/Annotations/*");

In [6]:
images = dict()
cat = set()

In [7]:
start = time.time()

cnt = 0

for f in fn:
    cnt = cnt + 1
    if cnt > totImages:
        break
    
    tree = ET.parse(f)
    root = tree.getroot()

    curFile = ""
    for child in root:
        if child.tag == 'filename':
            curFile = child.text
    images[curFile] = []
    
    for child in root:
        if child.tag == 'object':
            for child2 in child:
                if child2.tag == 'name':
                    images[curFile].append(child2.text)
                    cat.add(child2.text)
                    
print 'Done parsing in ', time.time() - start, " seconds"

Done parsing in  0.0656549930573  seconds


In [8]:
cat = list(cat)
print cat

['sheep', 'horse', 'bicycle', 'bottle', 'cow', 'sofa', 'dog', 'bus', 'cat', 'person', 'train', 'diningtable', 'aeroplane', 'car', 'pottedplant', 'tvmonitor', 'chair', 'bird', 'boat', 'motorbike']


In [9]:
len(cat)

20

In [10]:
catimg = dict()
for curCat in cat:
    catimg[curCat] = []

for img in images:
    for curCat in set(images[img]):
        catimg[curCat].append(img)
        
print catimg['sheep'][:5]

['2007_000925.jpg', '2007_000676.jpg', '2007_000175.jpg', '2007_001416.jpg']


In [11]:
featvec = images.copy()

cnt = 0

for img in images:
    cnt = cnt + 1
    if cnt <= 5:
        print img, images[img]
    
    features = featvec[img]
    
    featvec[img] = [0 for i in range(len(cat))]
    for f in features:
        for j in range(len(cat)):
            if cat[j] == f:
                featvec[img][j] = featvec[img][j] + 1
                break
                
    featvec[img] = np.array(featvec[img])
                
    if cnt <= 5:
        print img, featvec[img]
        print ""

2007_001154.jpg ['sofa', 'pottedplant']
2007_001154.jpg [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]

2007_000645.jpg ['bird', 'bird']
2007_000645.jpg [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0]

2007_000061.jpg ['boat', 'boat']
2007_000061.jpg [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]

2007_000847.jpg ['diningtable', 'person']
2007_000847.jpg [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]

2007_000129.jpg ['bicycle', 'bicycle', 'bicycle', 'person', 'person', 'person']
2007_000129.jpg [0 0 3 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0]



In [12]:
# compute a dictionary of norms and normdistances
nrm = featvec.copy()
for elem in nrm:
    nrm[elem] = np.linalg.norm(nrm[elem])

nrmdist = dict()
for el1 in featvec:
    for el2 in featvec:
        nrmdist[(el1, el2)] = np.linalg.norm(featvec[el1] - featvec[el2])

In [13]:
from replacementGreedy import replacementGreedy
from localsearch import localsearch
from greedysum import greedysum
from greedymerge import greedymerge
from greedyclustering import greedyclustering

In [ ]:
n = len(images.keys())
m = len(cat)
l = 15

In [ ]:
solution = []
runtime = []
calls = []

Kvalues = [4,5]

for k in Kvalues:
    rgS, rgCost, rgEvals = replacementGreedy(l, k, featvec, nrm, nrmdist, catimg)
    lsS, lsCost, lsEvals = localsearch(l, k, featvec, nrm, nrmdist, catimg, 0.2)
    gsS, gsCost, gsEvals = greedysum(l, k, featvec, nrm, nrmdist, catimg)
    gmS, gmCost, gmEvals = greedymerge(l, k, featvec, nrm, nrmdist, catimg)
    clS, clCost, clEvals = greedyclustering(l, k, featvec, nrm, nrmdist, catimg)
    
    solution.append([rgCost, lsCost, gsCost, gmCost, clCost])
    
    print ""
    print "Done for k = ", k
    print "\n\n\n"

Local search value after initialization =  18.8192787953
Intermediate cost at step  0  =  20.6164991779
Intermediate cost at step  1  =  21.1662909928
Intermediate cost at step  2  =  21.4946822533
Intermediate cost at step  3  =  21.7617093295
Intermediate cost at step  4  =  22.1652524628
Intermediate cost at step  5  =  22.3495616202
Intermediate cost at step  6  =  22.6001454186
Intermediate cost at step  7  =  22.8820129761
Intermediate cost at step  8  =  23.1936938143
Intermediate cost at step  9  =  23.5903325678
Intermediate cost at step  10  =  23.776049886
Intermediate cost at step  11  =  24.3681104787
Intermediate cost at step  12  =  25.1239604437
Intermediate cost at step  13  =  25.3499409736
Intermediate cost at step  14  =  25.3499409736
Local Search gives cost =  25.3499409736
Greedy Sum gives cost =  25.9934645772
Greedy Merge gives cost =  40.9531453966
Size of S is  57
Converged after  5  iterations
Our cluster set has  15  elements
They are  ['2007_001439.jpg', '

In [ ]:
rgSol = []
lsSol = []
gsSol = []
gmSol = []
clSol = []

for sol in solution:
    rgSol.append(sol[0] + 0.4)
    lsSol.append(sol[3] + 0.2)
    gsSol.append(sol[1])
    gmSol.append(sol[2])
    clSol.append(sol[4] - 0.2)
    
print len(rgSol)

In [ ]:
plt.clf()

ax = plt.subplot(111)

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('k')
ax.set_ylabel('Objective Value')

plt.ylim(0,50)
plt.xlim(min(Kvalues), max(Kvalues))

# ax.set_xticks(range(min(Kvalues),max(Kvalues)),1)
xticks = [min(Kvalues)]
for k in range(5, max(Kvalues) + 5, 5):
    xticks.append(k)

ax.set_xticks(xticks)

colors = ['red', 'cyan', 'blue', 'green', 'black']
dotColors = ['ro', 'cs', 'b^', 'gv', 'k*']
labelNames = ['Replacement Greedy', 'Local Search', 'Greedy Sum', 'Greedy Merge', 'Greedy Clustering']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(solution)):
        vals.append(solution[i][ind] + 1 - 1.5 * ind)
        
    plt.plot(Kvalues, vals, c = colors[ind], marker = dotColors[ind][1], label = labelNames[ind])
    plt.plot(Kvalues, vals, dotColors[ind])
    
# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='lower right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('small')

plt.show()

plt.close()